In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
data_dir="C:/Users/jeuux/Desktop/Carrera/MoAI/TFM/Data"
participants=[folder for folder in os.listdir(data_dir) if not(folder==".vscode")]
#create list with json paths
json_list=[os.path.join(data_dir,participant,"participant.json") for participant in participants ]

In [ ]:
def get_subject(json_file):
    with open(json_file) as f:
        participant_info = json.load(f)
    return participant_info["pa_info"]["Name"]
def save_participant_table(table,acc_path,current,count):
    #save table
    participant_folder=os.path.join(acc_path,current)

    if(not(os.path.isdir(participant_folder))):
        os.makedirs(participant_folder)

    save_path=os.path.join(participant_folder,"acc_{0}.csv".format(count))
    table.to_csv(save_path,index=False)
    print("Saved")

In [ ]:
#for each folder, read "participant.json" get participant name
names_list=[]
for json_file in json_list:
     names_list.append(get_subject(json_file))
    

In [3]:
#Getting Accelerometer Data
table_path="C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\Vermeersch data\\vermeersch Data Export - FULL TO DATE.tsv"
#Extract only columns we need
# cols=['Recording timestamp','Participant name','Accelerometer X','Accelerometer Y','Accelerometer Z',"Gyro X","Gyro Y", "Gyro Z"]
# df_chunck=pd.read_csv(table_path,delimiter='\t',usecols=cols,chunksize=150000)

In [4]:
cols=['Gaze point 3D X', 'Gaze point 3D Y', 'Gaze point 3D Z']
df_chunck=pd.read_csv(table_path,delimiter='\t',usecols=cols,chunksize=150000)


# Getting raw accelerometer tables

In [ ]:
acc_path="C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\Accelerometer_Data"
#Create tables with acc data of participants we have
for i,table in enumerate(df_chunck):
    #drop empty rows
#     table=table.dropna()
    #get participant names in batch
    names=table["Participant name"].unique()
    
    #Only one participant & is in our list
    if((len(names)==1)and names[0] in names_list):
        current=names[0]#current participant
        save_participant_table(table,acc_path,current,i)
        
    #More than one participant & some in our list
    elif((len(names)>1) and any(i in names_list for i in names)):
        #case 2
        partition={key: table.loc[value] for key, value in table.groupby('Participant name').groups.items()}
        for participant in partition.keys():
            if participant in names_list:
                save_participant_table(partition[participant],acc_path,participant,i)
        
        
        


# Create def tables (by concatenation)

In [ ]:
acc_path="C:\\Users\\jeuux\\Desktop\\Carrera\\MoAI\\TFM\\AnnotatedData\\Accelerometer_Data"
#concat
acc_folders=[folder for folder in os.listdir(acc_path)]

for folder in acc_folders:
    tables=[pd.read_csv(os.path.join(acc_path,folder,file),index_col=None) for file in os.listdir(os.path.join(acc_path,folder))] 
    full_table=pd.concat(tables)
    full_table=full_table.drop_duplicates()
    full_table=full_table.sort_values("Recording timestamp")
    full_table.to_csv(os.path.join(acc_path,folder,"acc_data{0}.csv".format(folder)),index=False)